# Crawl CNA news from several categories

Only 5 pieces of news on the first page from each category

中央社新聞各類新聞各5篇


In [1]:
import requests
from bs4 import BeautifulSoup
import re

# There are 11 categories in CNA news

Crawl links on a category's first page.

中央社新聞爬蟲-11類別的新聞(只能爬第一頁的內容，沒有翻頁的功能)

In [2]:
!pip install fake_useragent

## Crawl news for 11 categories

In [3]:
# 11 news categories
#news_links =['aipl', 'ait', 'aspt', 'asc', 'aie', 'amov','ahel','aopl','asoc','acul','acn']
#news_categories=['政治','科技','運動','證卷','產經','娛樂','生活','國際','社會','文化','兩岸']
news_links =['aipl', 'ait', 'aspt']
news_categories=['政治','科技','運動']
base_url = 'https://www.cna.com.tw/list/'

In [4]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta
from fake_useragent import UserAgent

In [5]:
user_agent = UserAgent()

In [ ]:
%%time
# 存放資料之變數
links = []
titles = []
dates = []
contents = []
categories = []
item_id = []
photo_links = []


# 這裡我們需要用for迴圈爬取11個類別
# Here we need to crawl 11 categories using for loop
for i, url_short_name in enumerate(news_links):  #針對每一類 共有11類

    category = news_categories[i] #'政治' #類別名稱紀錄起來 

    # Categorical url link
    category_url = base_url + url_short_name + '.aspx'
    print("Getting categorical news:", category)

    # Request the categorical news page
    # req = requests.get(category_url)
    req = requests.get(category_url, headers={ 'user-agent': user_agent.random }, timeout=5)
    page = BeautifulSoup(req.text, 'lxml')

    # 抓新聞列表
    items = page.find('ul', {'id': re.compile("jsMainList\w*")}).findAll('li')
    #print(items)


    # Let's start to crawl the news in the first page for that category
    serial_no = 1
    for item_j, item in enumerate(items,start=1): #針對每一篇項目 抓其細節
        title = item.find('h2').text
        print(serial_no,'--', title )
        
        link = item.find('a').get('href')
        print(link)

        try:
            if item.find('img').has_attr('data-src'):
                photo_link = item.find('img').get('data-src')
            else:
                photo_link = item.find('img').get('src')
        except:
            photo_link = ''
        
        # get date and convert it with the format of  "2022-04-20"
        news_date = item.find('div', {'class':"date"}).text
        print(news_date)
        dtime = datetime.strptime(news_date, '%Y/%m/%d %H:%M')
        # conver to the format of  "2022-04-20"
        news_date_str = dtime.strftime("%Y-%m-%d") # keep only date, remove the time.

        categories.append(category)
        titles.append(title)
        links.append(link)
        photo_links.append(photo_link)
        dates.append(news_date_str)
 
        # item_id.append("cna_" + tstr + "_" + str(serial_no))
        # conver to the format of  "20220420"
        dtstr = dtime.strftime("%Y%m%d") # keep only date, remove the time.
        # format of serial number: aipl_20220314_1
        
        item_id.append(url_short_name + "_" + dtstr + "_" + str(serial_no))
        serial_no += 1

        #抓新聞內容
        # 最新的新聞列表之新聞連結:開頭缺少"https://www.cna.com.tw"
        if link.startswith("/news"):
            link = "https://www.cna.com.tw"+link
        
        print(link)
        req = requests.get(link, headers={ 'user-agent': user_agent.random }, timeout=5)
        
        page = BeautifulSoup(req.text,'lxml')

        # Find content
        content = page.find('div',{'class':"paragraph"})

        # Remove unnecessary content
        # There are some words we don't like to analyze, They should be removed.
        # Remove "透過 Google News追蹤中央社"
        try:
            content.find('div', {'class':"gmailNews"}).decompose()
        except:
            pass
        # 額外插入的延伸閱讀區塊-->移除之
        try:
            content.find('div', {'class':"paragraph moreArticle"}).decompose()
        except:
            pass
        # 額外插入的訂閱連結-->移除之
        try:
            content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
        except:
            pass

        # 額外插入的圖片與文字說明區塊-->移除之
        try:
            for media in content.findAll('div', {'class':"media"}):
                media.decompose()
        except:
            pass

        # Remove unnecessary content
        cont = content.text
        cont = re.sub('（中央社.+?）','', cont)
        cont = re.sub('（編輯：.*','', cont) 
        cont = re.sub('（譯者：.*','', cont) 
        
        contents.append(cont)

        if item_j >= 4: # Here we crawl only 4 pieces of news for each category, in order to save time.
            break 
        
        time.sleep(10)  # 遵守爬蟲禮節，請小睡一下


Getting categorical news: 政治
1 -- 民團告發傅崐萁涉外患與強制罪 高檢署：依法辦理
/news/aipl/202503030336.aspx
2025/03/03 21:17
https://www.cna.com.tw/news/aipl/202503030336.aspx
2 -- 罷免案32藍委一階過關 吳思瑤等12綠委待補件
/news/aipl/202503030276.aspx
2025/03/03 18:18
https://www.cna.com.tw/news/aipl/202503030276.aspx
3 -- 美媒：台灣與捷克共築生命線 中俄壓力下加強合作
/news/aipl/202503030274.aspx
2025/03/03 18:17
https://www.cna.com.tw/news/aipl/202503030274.aspx
4 -- 藍委擬幫國軍加薪  國防部：應依法徵詢行政院
/news/aipl/202503030260.aspx
2025/03/03 17:54
https://www.cna.com.tw/news/aipl/202503030260.aspx
Getting categorical news: 科技
1 -- 美國太空飛行器「藍色幽靈」登陸月球 第2家民營企業締壯舉[影]
/news/ait/202503020141.aspx
2025/03/02 17:16
https://www.cna.com.tw/news/ait/202503020141.aspx
2 -- 雲達、技嘉揮軍MWC 展示AI與邊緣運算應用
/news/ait/202503030310.aspx
2025/03/03 19:41
https://www.cna.com.tw/news/ait/202503030310.aspx
3 -- 歐洲追求太空雄心 阿利安6型火箭將執行首趟商業飛行
/news/ait/202503030277.aspx
2025/03/03 18:20
https://www.cna.com.tw/news/ait/202503030277.aspx
4 -- 美國聯邦航空總署開綠燈 SpaceX星艦將第8度試射
/news/ait/202503030223.asp

In [14]:
titles

['民團告發傅崐萁涉外患與強制罪 高檢署：依法辦理',
 '罷免案32藍委一階過關 吳思瑤等12綠委待補件',
 '美媒：台灣與捷克共築生命線 中俄壓力下加強合作',
 '藍委擬幫國軍加薪  國防部：應依法徵詢行政院',
 '美國太空飛行器「藍色幽靈」登陸月球 第2家民營企業締壯舉[影]',
 '雲達、技嘉揮軍MWC 展示AI與邊緣運算應用',
 '歐洲追求太空雄心 阿利安6型火箭將執行首趟商業飛行',
 '美國聯邦航空總署開綠燈 SpaceX星艦將第8度試射',
 '許基宏5年約平均月薪80萬 王威晨李振昌獲複數年約',
 '美職運動家下放7人 莊陳仲敖林家正回小聯盟春訓',
 '悍將國際球探羅力加入春訓  9日安排球迷見面會',
 '首屆總統盃足球賽4/21開踢 鼓勵成年愛好者參與']

In [15]:
contents

['經民連與台灣東社今天分別告發國民黨立委傅崐萁涉犯外患罪、反滲透法與強制罪；高檢署表示，已收到告發狀，後續依法辦理。至晚間9時止，尚無法取得傅崐萁對此事的回應。台灣經濟民主連合智庫召集人賴中強等人今天在台灣高等檢察署前舉行記者會，現場發放的新聞資料指稱中國國民黨立法院黨團總召集人傅崐萁未獲行政院授權，2月到香港與多名香港區議員、香港旅遊業監管局官員等會商，宣稱代表「台灣的中央政府」，承諾台灣政府願提供補助進行在台跨縣市觀光合作。經民連說，傅崐萁去年率國民黨團17名成員訪問中國，會見中國人民政治協商會議全國委員會主席王滬寧等官員，返台後即遊說行政院官員等，開放福建平潭與台灣間海上客運直航復航。另外，傅崐萁昨天在花蓮一場活動中致詞時，近80歲的退休教師葉春蓮起身「嗆聲」被請出場，引發許多民眾及曾被葉春蓮教過的學生不滿。台灣東社發言人陳財能上午到花蓮縣警察局花蓮分局軒轅派出所，告發傅崐萁等違反中華民國刑法強制罪，要求警方做好證據保全及即刻調查，維護花蓮人人身自由及權利。',
 '中選會說，今天新增13件立委罷免案一階過關，另3件未達門檻。據了解，通過者皆為國民黨立委，未通過者為吳思瑤等3名民進黨立委。截至今天，共32件國民黨立委罷免案可進入二階連署，12件民進黨立委罷免案未過門檻，待補件。據了解，中選會最新一波審核通過名單，包括國民黨立委林德福、張智倫、楊瓊瓔、洪孟楷、羅明才、徐巧芯、賴士葆、廖先翔、徐欣瑩、林思銘、江啟臣、羅智強、謝衣鳯共13人。至於3件未達提議門檻為民進黨立委吳思瑤、吳沛憶與陳瑩罷免案，補提以1次為限。除此之外，先前經中選會一階審核通過名單包括國民黨立委王鴻薇、李彥秀、葉元之、牛煦庭、涂權吉、魯明哲、萬美玲、呂玉玲、邱若華、鄭正鈐、游顥、馬文君、顏寬恒、廖偉翔、黃健豪、羅廷瑋、丁學忠、黃建賓、傅崐萁共19人，以及遭停職的新竹市長高虹安，另還有民進黨南投縣議員蔡銘軒、陳玉鈴。先前未通過的9案則為民進黨立委伍麗華、張宏陸、蘇巧慧、李坤城、蔡其昌、何欣純、陳冠廷、林俊憲、王定宇，尚待補件。目前立委罷免案方面，僅剩無黨籍立委陳超明；國民黨立委邱鎮軍、林沛祥；民進黨立委黃捷、許智傑、王美惠、吳琪銘與陳俊宇結果尚未出爐。多個罷免團體趕在公職人員選舉罷免法修正條文2月20日生效前，遞交第一階段罷免提議書。中選會共收到64案罷免案送件，其中54案為立委（部分為罷免對

# Save data

In [9]:
data = zip(item_id, dates, categories, titles, contents, links, photo_links)
df = pd.DataFrame(list(data), columns=['item_id','date','category','title','content','link','photo_link'])

In [10]:
df.head(2)

item_id        date category                    title  \
0  aipl_20250303_1  2025-03-03       政治  民團告發傅崐萁涉外患與強制罪 高檢署：依法辦理   
1  aipl_20250303_2  2025-03-03       政治  罷免案32藍委一階過關 吳思瑤等12綠委待補件   

                                             content  \
0  經民連與台灣東社今天分別告發國民黨立委傅崐萁涉犯外患罪、反滲透法與強制罪；高檢署表示，已收到...   
1  中選會說，今天新增13件立委罷免案一階過關，另3件未達門檻。據了解，通過者皆為國民黨立委，未...   

                           link  \
0  /news/aipl/202503030336.aspx   
1  /news/aipl/202503030276.aspx   

                                          photo_link  
0  https://imgcdn.cna.com.tw/www/WebPhotos/200/20...  
1  https://imgcdn.cna.com.tw/www/webphotos/WebCov...

In [11]:
df.shape

(12, 7)

In [12]:
df.content[0]

'經民連與台灣東社今天分別告發國民黨立委傅崐萁涉犯外患罪、反滲透法與強制罪；高檢署表示，已收到告發狀，後續依法辦理。至晚間9時止，尚無法取得傅崐萁對此事的回應。台灣經濟民主連合智庫召集人賴中強等人今天在台灣高等檢察署前舉行記者會，現場發放的新聞資料指稱中國國民黨立法院黨團總召集人傅崐萁未獲行政院授權，2月到香港與多名香港區議員、香港旅遊業監管局官員等會商，宣稱代表「台灣的中央政府」，承諾台灣政府願提供補助進行在台跨縣市觀光合作。經民連說，傅崐萁去年率國民黨團17名成員訪問中國，會見中國人民政治協商會議全國委員會主席王滬寧等官員，返台後即遊說行政院官員等，開放福建平潭與台灣間海上客運直航復航。另外，傅崐萁昨天在花蓮一場活動中致詞時，近80歲的退休教師葉春蓮起身「嗆聲」被請出場，引發許多民眾及曾被葉春蓮教過的學生不滿。台灣東社發言人陳財能上午到花蓮縣警察局花蓮分局軒轅派出所，告發傅崐萁等違反中華民國刑法強制罪，要求警方做好證據保全及即刻調查，維護花蓮人人身自由及權利。'

In [13]:
df.to_csv("cna_category_news.csv", sep="|", index=False)